# Parsing JSON Files of Scraped Amazon Reviews

In [175]:
#importing dependencies
import json
import pandas as pd
import os
import glob

In [203]:
#importing file names for json's not processed yet

def json_extract(path):
    json_list=[]
    extension = 'json'
    os.chdir(path)
    json_list = [i for i in glob.glob('*.{}'.format(extension))]
    print(json_list)
    return json_list 

json_files = json_extract('/Users/samk/Box/scrapers/amazon.com/AmazonScraper/Departments/')


['ProductReviews_beauty.json', 'ProductReviews_groceries.json']


In [216]:
#loading json's
with open('ProductReviews_groceries.json') as json_file:  
    data = json.load(json_file)

In [217]:
#Checking the length of the data
len(data)

8345

In [207]:
# Raw data layout
data[0]

{'asin': ['B01LSUQSB0'],
 'reviewers_id': ['Chris',
  'micki j.',
  'Elizabet',
  'Maria Andrea Espiritu',
  'sewgr8nct',
  'Mila',
  'Tampa mom',
  'Amazon Customer',
  'Tim Maurer'],
 'reviews_rating': ['5.0 ',
  '1.0 ',
  '5.0 ',
  '1.0 ',
  '5.0 ',
  '1.0 ',
  '1.0 ',
  '1.0 ',
  '1.0 '],
 'reviews_title': ['Like, wow',
  'Poor product -- Poor Amazon Prime customer care',
  'Must have!',
  'WORST PURCHASE IN MY LIFE',
  'Best hair tool ever invented!!',
  'WARNING',
  'Loved it, until the motor burnt out on 3rd use',
  'Be Aware of malfunction in a short period of time',
  'Used it the 1st time and absolutely loved it! I was ready to share with all ...'],
 'reviews_date': ['March 21, 2018',
  'July 25, 2017',
  'December 15, 2017',
  'May 24, 2018',
  'March 30, 2017',
  'September 15, 2018',
  'September 25, 2017',
  'May 8, 2018',
  'May 22, 2017'],
 'reviews_text': [['I have a lot of hair. A LOT. Like, wow, who is that Wookiee over there LOTS. I have looked into having a third a

In [218]:
#Function parsing extracting product info from scraped lists and appending it to new list (all lists have equal range)
def parse_reviews(review):
    asin= review['asin'][0]
    reviewers_id = []
    review_rating = []
    review_title = []
    review_date = []
    review_text = []
    for i in range(0, len(review['reviewers_id'])):
        ids = review['reviewers_id'][i]
        rating = review['reviews_rating'][i]
        title = review['reviews_title'][i]
        date = review['reviews_date'][i]
        text = review['reviews_text'][0][i]
        reviewers_id.append(ids)
        review_rating.append(rating)
        review_title.append(title)
        review_date.append(date)
        review_text.append(text)
        
    return asin, reviewers_id, review_rating, review_title, review_date, review_text
       
        
    


In [219]:
#Parsing for review page extract
parse_reviews(data[0])

('B0099HD3YA',
 ['J. Brazalovich',
  'Josh F',
  'Amazon Customer',
  'Romain',
  'Lee',
  'Megh',
  'Rosie',
  'B. Blom',
  'PB&J'],
 ['5.0 ', '5.0 ', '1.0 ', '1.0 ', '1.0 ', '5.0 ', '1.0 ', '1.0 ', '1.0 '],
 ['Nespresso is Bestpresso',
  "Good Representation, Decide What's Right for You",
  'DO NOT BUY THESE! LISTEN TO ALL OF THE NEGATIVE REVIEWS!',
  '... this shop (almost monthly) this year and it was fine at the beginning',
  'Broken pods in every order',
  "They're Simply the Best Capsules for the Nespresso Machines - Just Buy Them Already",
  'Great Product,  capsule are FRAGILE, handle carefully.  And hope your pkging is better than what came to me.',
  'Cups came crushed!',
  "Don't Waste Your Money - Pods Bent/Crushed and More Expensive Than from Nespresso"],
 ['June 3, 2016',
  'August 6, 2015',
  'January 20, 2018',
  'August 11, 2017',
  'March 1, 2017',
  'January 30, 2018',
  'January 19, 2017',
  'December 3, 2017',
  'February 21, 2018'],
 ["I tried to quit you, Nespre

In [220]:
# Parsing all pages extracted 
parsed = []
for i in range(0,len(data)):
    all_reviews = parse_reviews(data[i])
    parsed.append(all_reviews)

In [222]:
# Parsing each review with its metadata for every page
asin = []
reviewer_id = []
rating = []
title = []
date = []
text  = [] 
for i in range(0, len(parsed)):
    for j in range(0,9):
        asins = parsed[i][0]
        reviewer_ids = parsed[i][1][j]
        ratings = parsed[i][2][j]
        titles = parsed[i][3][j]
        dates = parsed[i][4][j]
        texts = parsed[i][5][j]
        asin.append(asins)
        reviewer_id.append(reviewer_ids)
        title.append(titles)
        rating.append(ratings)
        date.append(dates)
        text.append(texts)

In [223]:
# Dumping metadata into a dictionary 
parsed_to_dict = {'asin': asin,'id':reviewer_id,'rating':rating, 'title':title, 'date': date,'text': text}

In [224]:
#Checking the len of the dictionary keys
print(len(asin), len(reviewer_id), len(rating), len(title), len(date), len(text))

75105 75105 75105 75105 75105 75105


In [225]:
# Dumping dictionary into dataframe (each line is a review for a specific product that can be retrieved by its ASIN)
parsed_df = pd.DataFrame.from_dict(parsed_to_dict)
parsed_df

,asin,id,rating,title,date,text
0,B0099HD3YA,J. Brazalovich,5.0,Nespresso is Bestpresso,"June 3, 2016","I tried to quit you, Nespresso. I tried cheap..."
1,B0099HD3YA,Josh F,5.0,"Good Representation, Decide What's Right for You","August 6, 2015","I just bought my Nespresso Inissia online, and..."
2,B0099HD3YA,Amazon Customer,1.0,DO NOT BUY THESE! LISTEN TO ALL OF THE NEGATIV...,"January 20, 2018",I should have listened to the other negative r...
3,B0099HD3YA,Romain,1.0,... this shop (almost monthly) this year and i...,"August 11, 2017",I made regular purchases from this shop (almos...
4,B0099HD3YA,Lee,1.0,Broken pods in every order,"March 1, 2017",We really love Nespresso. We order Nespresso p...
5,B0099HD3YA,Megh,5.0,They're Simply the Best Capsules for the Nespr...,"January 30, 2018",I tried to buy the cheaper knock-off capsules ...
6,B0099HD3YA,Rosie,1.0,"Great Product, capsule are FRAGILE, handle ca...","January 19, 2017",1/27/17. Another star gone. product was great...
7,B0099HD3YA,B. Blom,1.0,Cups came crushed!,"December 3, 2017",I am so disappointed in this product. It arriv...
8,B0099HD3YA,PB&J,1.0,Don't Waste Your Money - Pods Bent/Crushed and...,"February 21, 2018",Ten (20%!) of the pods came slightly to mostly...
9,B00ID0OWUS,Charissa013,5.0,Love the variety pack,"June 20, 2016",These are the best granola bars out there. Lov...


In [227]:
#Writing out the dataframe into a csv file
parsed_df.to_csv('Groceries_reviews.csv')